In [61]:
from pdfminer import high_level
from PIL import Image
import fitz
import io
from fpdf import FPDF
import re
from datetime import datetime

In [75]:
#Acquiring holder information in text from PDF.
path = r'C:\Users\Yatharth\Documents\VacCert-Project\certs\navya.pdf' #filepath: will change on deployment
text = high_level.extract_text(path)
#text is raw data from pdf, print text to view extraction
#print(text)
text = text.replace("\n\n", "%")
#print(text)
ls = text.split("%")
#print(ls)
sep = ["Beneﬁciary", "Final"]
inx = 0
for entry in ls:
    if entry.split(' ',1)[0] == sep[0] or entry.split(' ',1)[0] == sep[1]:
        break
    inx = inx + 1
ls = ls[1:]
#print(inx)
#print(ls)
#ls is list with proper entries from PDF
#dictionary for storing holder information, in order of how it is in ls
info = ["name", "age", "sex", "verif", "uhid", "refID", "vax", "dose1", "dose2"]
info_dict = {}

#basic info (name,age,sex,verification), hopefully this is all in order
for i in range(4):
    info_dict[info[i]] = ls[i]

#print(info_dict)

#print(len(ls[]))
doses = []
for lsen in ls:   
    if len(lsen) > 2 and lsen[2] == '-' and "uhid" not in info_dict:
        info_dict["uhid"] = lsen
    elif len(lsen) == 14 and lsen.isdigit() and "refID" not in info_dict:
        info_dict["refID"] = lsen
    elif "vax" not in info_dict and ("COVISHIELD" in lsen or "COVAXIN" in lsen or "SPUTNIK" in lsen):
        info_dict["vax"] = lsen
    elif len(lsen) > 3 and lsen[0:2].isdigit() and lsen[3].isalpha():
        ix = lsen.find("(")
        date = lsen[:ix-1]
        date = datetime.strptime(date, "%d %b %Y")
        doses.append(date)
        
doses = sorted(doses)
zipped = zip(info[7:],doses)

for doseno,dosedt in zipped:
    info_dict[doseno] = dosedt.strftime('%d %B %Y')

print(info_dict)

#info_dict has all holder information as a dict

{'name': 'Navya Ambardar', 'age': '23', 'sex': 'Female', 'verif': 'Aadhaar # XXXXXXXX2129', 'refID': '51712044174060', 'vax': 'COVISHIELD', 'dose1': '03 July 2021', 'dose2': '04 October 2021'}


In [76]:
#Acquiring QR code from PDF. Please leave untouched, it works as expected.
#Opening with fitz
pdf_file = fitz.open(path)
page = pdf_file[0] #only page in the pdf
image_list = page.getImageList() 
xref = image_list[0][0]
base_image = pdf_file.extractImage(xref)
image_bytes = base_image["image"]
image_ext = base_image["ext"]
qr = Image.open(io.BytesIO(image_bytes))
qr.save(open(f"qr.{image_ext}", "wb"))

In [77]:
#the QR has to stay this way. Several problems on the way, but we have to accomodate, since FPDF is not perfect
qr_link = r'C:\Users\Yatharth\Documents\VacCert-Project\vaccine-certificate-shortener\qr.png'
hfw_img = r'C:\Users\Yatharth\Documents\VacCert-Project\vaccine-certificate-shortener\imgfiles\mohfw.png'

In [78]:
#Creating new certificate with PyFPDF
ph_w = 281.25
ph_h = 500.25
ln_y = 274
ln_x1 = 21
ln_x2 = 260
def mob_1():
    pdf = FPDF('P','pt', (ph_w,ph_h))
    pdf.add_page()
    pdf.set_margins(left=0, top=0, right=0)
    pdf.image(hfw_img,x=75,y=24.5,w=132,h=68,type="png")
    pdf.set_font('Arial', 'B', 11.2)
    pdf.set_text_color(0,0,125)
    pdf.cell(w=225,h=166,txt="Final COVID-19 Vaccine Certificate",border=0,align='C',ln=1)
    pdf.set_line_width(0.1)
    pdf.line(ln_x1,ln_y,ln_x2,ln_y)
    pdf.image(qr_link,x=49,y=302,w=184,h=184,type="png")
    pdf.set_font('Arial', size=7)
    pdf.set_text_color(0,0,0) 
    pdf.ln(h=96)
    pdf.cell(w=282,txt="https://verify.cowin.gov.in",border=0,align='C',link="https://verify.cowin.gov.in",ln=2)
    return pdf

In [79]:
mob = mob_1()
mob.set_font('Arial', 'B',size=9)
mob.set_text_color(31,73,125)
info = "Vaccine \nBeneficiary Name \nAge \nGender \nVerified ID \nUHID \nReference ID \nVaccination Dates"
seq = ['vax','name','age','sex','verif','uhid','refID','dose1','dose2']
data = ''
for i in seq:
    if i in info_dict.keys():
        data = data + info_dict[i] + "\n"
    else:
        data = data + "\n"
mob.set_xy(x = 18,y = 127)
mob.multi_cell(w=250,h=15,txt=info,align='L',border=0)
mob.set_xy(x = 18,y = 127)
mob.set_font('Arial',size=9)
mob.set_text_color(0,0,0)
mob.multi_cell(w=242,h=15,txt=data,align='R',border=0)
mob.output('mobile1.pdf', 'F')


''